# Main script to Join Data on Instrument 2 Fire Location

Modules: N/A <br>
Author: Jordan Meyer <br>
Email: jordan.meyer@berkeley.edu <br>
Date created: Feb 18, 2023 <br>

**Citations (data sources)**


**Citations (persons)**
1. Cornelia Ilin 

**Preferred environment**
1. Code written in Jupyter Notebooks

### Step 1: Import packages

In [1]:
import pandas as pd
import numpy as np
import netCDF4 as ncdf
import os
from datetime import date, timedelta
from math import pi
import fiona

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

# geography
import geopandas as gpd
import osmnx as ox
import shapely
from shapely.geometry import Point
import contextily as ctx 

#Moved from sklearn.neighbors to sklearn.metrics following their package change
import sklearn.metrics
dist = sklearn.metrics.DistanceMetric.get_metric(
    'haversine'
)

# ignore warnings
import warnings
warnings.filterwarnings(
    'ignore'
)


### Step 2: Define working directories

In [2]:
#Local directories on my machine (not gdrive)
in_dir = '../data/processed/'
out_dir = './'

In [3]:
ls

1.0-jam-wind-cleaning.ipynb
1.1-jam-wind-direction.ipynb
1.2-jam-wind-functions.ipynb
3.0-jmm-fires_cleaning.ipynb
3.1-jmm-fires_clean_eda.ipynb
3.2-jmm-fires_dropped_eda.ipynb
3.3-jam-fire_zcta_add.ipynb
3.4-jam-fires_final_stage_cleaning_zctas.ipynb
3.5-jam-fire-trim-multipolygons.ipynb
4.0-jam-join_data_instrument_2.ipynb


### Step 3: Define functions

### Step 4: Read data

## Wind

Wind load from other file

In [4]:
wind_df = pd.read_csv(in_dir + "all_years_wind_data.csv",index_col=0).sort_values(["year_month","ZCTA10"]).reset_index(drop=True)

wind_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/all_years_wind_data.csv'

## Fire

Finsh cleaning some fire things and then join fire to wind 

In [300]:
fire_df = pd.read_csv(in_dir + "fire_zipcodes_compressed.csv",index_col=0)

#Extract year and month, filter for fires in scope 1991+
fire_df['year'] = fire_df['ALARM_DATE'].str[:4]
fire_df = fire_df[fire_df['year'].astype(int)>1990]


#Extract Month and End Year/Month
fire_df['month'] = fire_df['ALARM_DATE'].str[5:7]
fire_df['year_month'] = fire_df['year']+fire_df['month']
fire_df['month'] = fire_df['month'].astype(int)
fire_df['end_year'] = fire_df['CONT_DATE'].str[:4]
fire_df['end_month'] = fire_df['CONT_DATE'].str[5:7].astype(int)

#Calculate duration with monthly basis, add 1 as proxy for ceiling calculation ie fire at any time in month counts regardless of start date
fire_df['duration_months'] = (fire_df['end_month']-fire_df['month'])+1
fire_df = fire_df.set_index(['ZCTA','ALARM_DATE'])

# For fires that span multiple years replace the values of the duration on a yearly basis and duplciate the entry for the next year
for idx,fire in fire_df[fire_df['end_year'] != fire_df['year']].iterrows():
    fire['FIRE_NAME'] = fire['FIRE_NAME']+" CONT"
    fire['year'] = int(fire['year'])+1
    fire['month'] = "01"
    fire['year_month'] = str(fire['year']) + fire['month']
    #Hard coding duration = 1 as all fires that spanned across years happened to be put out in January
    fire['duration_months'] = 1
    
    #Replace the total durations with the duration in year, clear up negatives durations from month 2 < month 1
    fire_df.loc[idx,'duration_months'] =  (13-fire_df.loc[idx,'month'])
    
    #Append the new year's entry for the fire's continuation
    fire_df = fire_df.append(fire)

fire_df = fire_df.reset_index()

In [301]:
fire_df.shape

(6868, 18)

In [302]:
# For all fires that are longer than 1 month insert another entry with remaing duration into the dataframe
print(fire_df[fire_df['duration_months']>1].shape)
for idx,fire in fire_df[fire_df['duration_months']>1].iterrows():
    month = 2
    duration = fire['duration_months'] 
    while duration > 1:
        # print(idx, fire_df.iloc[idx]['duration_months'], duration-1)
        #fire['FIRE_NAME'] = fire_df.iloc[idx]['FIRE_NAME'] + f"_{month}"
        fire['duration_months'] -= 1
        fire['month'] += 1
        fire['year_month'] = (str(fire['year']) + str(fire['month'])) if len(str(fire['month']))>1 else (str(fire['year']) + "0" + str(fire['month'])) 
        fire_df = fire_df.append(fire)
        duration = fire['duration_months']
        month += 1

(1419, 18)


In [303]:
pd_fire = pd.DataFrame(fire_df.drop('fire_centroid',axis=1))

In [304]:
fire_df.drop_duplicates()

,ZCTA,ALARM_DATE,FIRE_NAME,CONT_DATE,CAUSE,GIS_ACRES,Shape_Length,Shape_Area,index_right,fire_centroid,DURATION,FIRE_AREA_KM2,year,month,year_month,end_year,end_month,duration_months
0,95648.0,2020-06-18,NELSON,2020-06-23,11.0,109.602280,3252.523280,4.435447e+05,405.0,POINT (-121.3480590211847 38.88804091206984),5 days,0.443546,2020,6,202006,2020,6,1
1,95747.0,2020-06-01,AMORUSO,2020-06-04,2.0,685.585022,9653.760308,2.774464e+06,1217.0,POINT (-121.3708983104108 38.82756661261951),3 days,2.774464,2020,6,202006,2020,6,1
2,95648.0,2020-03-31,FLEMING,2020-04-01,9.0,12.931545,1577.155857,5.233211e+04,405.0,POINT (-121.2734135751918 38.9623284462546),1 days,0.052332,2020,3,202003,2020,4,2
3,95966.0,2020-04-14,MELANESE,2020-04-19,18.0,10.315964,1035.787625,4.174722e+04,933.0,POINT (-121.3006534927401 39.48636412414794),5 days,0.041747,2020,4,202004,2020,4,1
4,95747.0,2020-07-05,PFE,2020-07-05,14.0,36.701931,2348.114043,1.485274e+05,1217.0,POINT (-121.3810176618852 38.73133921489409),0 days,0.148527,2020,7,202007,2020,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6822,95546.0,2016-08-22,TULLEY,2016-09-04,7.0,607.445740,13095.080608,2.458246e+06,277.0,POINT (-123.8140318155857 41.28541623395056),13 days,2.458246,2016,9,201609,2016,9,1
6847,96130.0,2016-07-30,DOCKERY,2016-08-17,10.0,27.963249,2655.210701,1.131633e+05,51.0,POINT (-120.6873444598079 40.42539576204376),18 days,0.113163,2016,8,201608,2016,8,1
6848,96015.0,2016-09-11,HOWARD,2016-10-17,6.0,379.622528,7898.270122,1.536278e+06,1021.0,POINT (-120.9157313359543 41.47071744410114),36 days,1.536278,2016,10,201610,2016,10,1
6849,96130.0,2016-09-11,WILLARD,2016-11-28,14.0,2573.955322,27364.478495,1.041643e+07,51.0,POINT (-120.7510806570093 40.37593783404107),78 days,10.416427,2016,10,201610,2016,11,2


In [353]:
pd_fire.drop_duplicates().set_index(['ZCTA','year_month'])

,,ALARM_DATE,FIRE_NAME,CONT_DATE,CAUSE,GIS_ACRES,Shape_Length,Shape_Area,index_right,DURATION,FIRE_AREA_KM2,year,month,end_year,end_month,duration_months
ZCTA,year_month,,,,,,,,,,,,,,,
95648,202006,2020-06-18,NELSON,2020-06-23,11.0,109.602280,3252.523280,4.435447e+05,405.0,5 days,0.443546,2020,6,2020,6,1
95747,202006,2020-06-01,AMORUSO,2020-06-04,2.0,685.585022,9653.760308,2.774464e+06,1217.0,3 days,2.774464,2020,6,2020,6,1
95648,202003,2020-03-31,FLEMING,2020-04-01,9.0,12.931545,1577.155857,5.233211e+04,405.0,1 days,0.052332,2020,3,2020,4,2
95966,202004,2020-04-14,MELANESE,2020-04-19,18.0,10.315964,1035.787625,4.174722e+04,933.0,5 days,0.041747,2020,4,2020,4,1
95747,202007,2020-07-05,PFE,2020-07-05,14.0,36.701931,2348.114043,1.485274e+05,1217.0,0 days,0.148527,2020,7,2020,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95546,201609,2016-08-22,TULLEY,2016-09-04,7.0,607.445740,13095.080608,2.458246e+06,277.0,13 days,2.458246,2016,9,2016,9,1
96130,201608,2016-07-30,DOCKERY,2016-08-17,10.0,27.963249,2655.210701,1.131633e+05,51.0,18 days,0.113163,2016,8,2016,8,1
96015,201610,2016-09-11,HOWARD,2016-10-17,6.0,379.622528,7898.270122,1.536278e+06,1021.0,36 days,1.536278,2016,10,2016,10,1


In [384]:
pd_fire['ZCTA'] = pd_fire['ZCTA'].astype(str).str[:5]
pd_fire['year_month'] = pd_fire['year_month'].str[:]


In [365]:
pd_fire.dtypes

ZCTA                object
ALARM_DATE          object
FIRE_NAME           object
CONT_DATE           object
CAUSE              float64
GIS_ACRES          float64
Shape_Length       float64
Shape_Area         float64
index_right        float64
DURATION            object
FIRE_AREA_KM2      float64
year                object
month               object
year_month          object
end_year            object
end_month            int64
duration_months     object
dtype: object

In [392]:
fire_ready = pd_fire[~pd_fire['ZCTA'].isna()]\
    .reset_index(drop=True)\
    .drop_duplicates()


In [393]:
wind_ready = wind_df\
    .drop('Unnamed: 0', axis=1)\
    .rename(columns={'ZCTA10':'ZCTA'})\
    .reset_index(drop=True)\
    .drop_duplicates()

In [405]:
wind_ready['year_month'] = wind_ready['year_month'].astype(str)
wind_ready['ZCTA'] = wind_ready['ZCTA'].astype(str)

In [407]:
wind_ready.dtypes[['year_month','ZCTA']]

year_month    object
ZCTA          object
dtype: object

In [408]:
fire_ready.dtypes[['year_month','ZCTA']]

year_month    object
ZCTA          object
dtype: object

In [411]:
wind_fire = wind_ready.merge(fire_ready, on=['year_month','ZCTA'], how='left')

In [412]:
wind_fire

,lat,lon,ZCTA,u,v,wdir,wspd,year_month,year_x,month_x,...,Shape_Length,Shape_Area,index_right,DURATION,FIRE_AREA_KM2,year_y,month_y,end_year,end_month,duration_months
0,37.465,-117.936,89010,0.504258,-0.719008,305.042938,0.878208,199101,1991,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,35.396,-116.322,89019,-0.172753,-0.946940,259.661102,0.962568,199101,1991,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36.161,-116.139,89060,-0.435964,-0.812957,241.796738,0.922477,199101,1991,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,35.957,-115.897,89061,-0.560538,-1.176384,244.522552,1.303105,199101,1991,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,39.520,-120.032,89439,0.042253,0.205661,78.390099,0.209957,199101,1991,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628502,39.149,-120.248,96146,0.636599,0.953782,56.279018,1.146716,202212,2022,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
628503,39.236,-120.062,96148,0.636599,0.953782,56.279018,1.146716,202212,2022,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
628504,38.732,-120.033,96150,0.071517,0.472164,81.387100,0.477549,202212,2022,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
628505,39.184,-120.427,96161,-0.217010,0.239125,132.224274,0.322915,202212,2022,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## PM2.5

Load PM2.5 from lawis files downloaded from gdrive

In [450]:
aqi1 = pd.read_csv(in_dir + '1990_to_1999_pm25.csv', index_col = 0)
aqi1.shape

(162720, 3)

In [451]:
aqi2 = pd.read_csv(in_dir + 'UW_pm25_zip_monthly_anand_2000-2018-v2.csv', index_col = 0)
aqi2.shape

(400368, 5)

In [452]:
aqi1 = aqi1.reset_index(drop=True)\
    .rename(columns={'ZCTA5CE10':'ZCTA',
                    'year_month':'old_ym'})

In [453]:
aqi2 = aqi2.reset_index(drop=True)\
    .rename(columns={'ZIP10':'ZCTA',
                    'year_month':'old_ym'})\
    .drop(['year_month_zip','GEOID10'],axis=1)

In [454]:
pm25_df = aqi1.append(aqi2)
pm25_df.head()

,old_ym,ZCTA,pm25
0,1990-01,89010,11.251951
1,1990-01,89439,8.439565
2,1990-01,90004,33.266667
3,1990-01,90006,34.833333
4,1990-01,90007,33.627143


In [449]:
pm25_df.shape

(563088, 3)

In [466]:
pm25_df['year_month']=pm25_df['old_ym'].str[:4]+pm25_df['old_ym'].str[5:]
pm25_df['ZCTA'] = pm25_df['ZCTA'].astype(str)
pm25_ready = pm25_df.drop('old_ym',axis=1)

In [467]:
pm25_ready.dtypes

ZCTA           object
pm25          float64
year_month     object
dtype: object

In [468]:
wfp_df = wind_fire.merge(pm25_ready, on=['year_month','ZCTA'], how='left')

In [471]:
wfp_df.head()

,lat,lon,ZCTA,u,v,wdir,wspd,year_month,year_x,month_x,...,Shape_Area,index_right,DURATION,FIRE_AREA_KM2,year_y,month_y,end_year,end_month,duration_months,pm25
0,37.465,-117.936,89010,0.504258,-0.719008,305.042938,0.878208,199101,1991,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.450976
1,35.396,-116.322,89019,-0.172753,-0.946940,259.661102,0.962568,199101,1991,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36.161,-116.139,89060,-0.435964,-0.812957,241.796738,0.922477,199101,1991,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,35.957,-115.897,89061,-0.560538,-1.176384,244.522552,1.303105,199101,1991,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,39.520,-120.032,89439,0.042253,0.205661,78.390099,0.209957,199101,1991,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.389565


In [478]:
wfp_df['treatment'] = ~wfp_df['duration_months'].isna()

In [487]:
print(f"Number of months/zips in treatment group {np.sum(wfp_df['treatment'])}")

Number of months/zips in treatment group 7584


In [486]:
print(f"Number of months/zips in control group {np.sum(~wfp_df['treatment'])}")

number of months/zips in control group 620923


In [488]:
wfp_df.columns

Index(['lat', 'lon', 'ZCTA', 'u', 'v', 'wdir', 'wspd', 'year_month', 'year_x',
       'month_x', 'ALARM_DATE', 'FIRE_NAME', 'CONT_DATE', 'CAUSE', 'GIS_ACRES',
       'Shape_Length', 'Shape_Area', 'index_right', 'DURATION',
       'FIRE_AREA_KM2', 'year_y', 'month_y', 'end_year', 'end_month',
       'duration_months', 'pm25', 'treatment'],
      dtype='object')

In [491]:
final_df = wfp_df.drop(['year_x','month_x', 'ALARM_DATE', 'FIRE_NAME', 
                        'CONT_DATE', 'CAUSE','Shape_Length', 'Shape_Area', 
                        'index_right','year_y', 'month_y','duration_months',
                        'end_year', 'end_month'],axis=1)

In [492]:
final_df

,lat,lon,ZCTA,u,v,wdir,wspd,year_month,GIS_ACRES,DURATION,FIRE_AREA_KM2,pm25,treatment
0,37.465,-117.936,89010,0.504258,-0.719008,305.042938,0.878208,199101,NaN,NaN,NaN,12.450976,False
1,35.396,-116.322,89019,-0.172753,-0.946940,259.661102,0.962568,199101,NaN,NaN,NaN,NaN,False
2,36.161,-116.139,89060,-0.435964,-0.812957,241.796738,0.922477,199101,NaN,NaN,NaN,NaN,False
3,35.957,-115.897,89061,-0.560538,-1.176384,244.522552,1.303105,199101,NaN,NaN,NaN,NaN,False
4,39.520,-120.032,89439,0.042253,0.205661,78.390099,0.209957,199101,NaN,NaN,NaN,8.389565,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
628502,39.149,-120.248,96146,0.636599,0.953782,56.279018,1.146716,202212,NaN,NaN,NaN,NaN,False
628503,39.236,-120.062,96148,0.636599,0.953782,56.279018,1.146716,202212,NaN,NaN,NaN,NaN,False
628504,38.732,-120.033,96150,0.071517,0.472164,81.387100,0.477549,202212,NaN,NaN,NaN,NaN,False
628505,39.184,-120.427,96161,-0.217010,0.239125,132.224274,0.322915,202212,NaN,NaN,NaN,NaN,False


In [493]:
final_df.to_csv('instrument2.csv')